In [6]:
from database.connect import getConnection
import JsonHandler as jh
import mysql
import pandas as pd

In [7]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

In [8]:
print(jh.languages_list)


['ar', 'en', 'fr', 'pt', 'sp', 'it', 'de', 'hi']


In [9]:
airlines_dict = {'KLM': ['klm'],
                'AirFrance':['airfrance',
                            'air france'],
                'British_Airways': ['british_airways',
                                    'british airways'],
                'AmericanAir': ['americanair',
                                'american airlines'],
                'Lufthansa': ['lufthansa'],
                'AirBerlin': ['airberlin',
                                'air berlin'],
                'AirBerlin assist': ['airberlin assist',
                                    'air berlin assist',
                                    'airberlinassist'],
                'easyJet': ['easyjet'],
                'RyanAir': ['ryanair'],
                'SingaporeAir': ['singaporeair',
                                'singapore airlines'],
                'Qantas': ['qantas'],
                'EtihadAirways': ['etihad airways',
                                'etihadairways',
                                'etihad'],
                'VirginAtlantic': ['virgin atlantic',
                                    'virginatlantic'],
            }

In [10]:
# Create a cursor to execute SQL queries
cursor = connection.cursor()

for key, value in airlines_dict.items():
    query = f"""
        SELECT
            t.language AS Language,
            (SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS ParticipatedPercentage,
            COUNT(*) AS TotalConversations,
            SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) AS ParticipatedConversations
        FROM
            conversations c
            INNER JOIN part_of p ON c.id = p.cID
            INNER JOIN tweets t ON t.id = p.tID
        WHERE
            (t.airlines LIKE '%lazy_query_strat_ignore_this%'
    """
    for airline in value:
        query += f" OR t.airlines LIKE '%{airline}%'"

    query += """
        )
        GROUP BY Language
    """

    cursor.execute(query)
    result = cursor.fetchall()
    print(key)
    print(result)





KLM
[('ar', Decimal('8.0000'), 25, Decimal('2')), ('de', Decimal('62.5000'), 240, Decimal('150')), ('en', Decimal('56.2569'), 13673, Decimal('7692')), ('fr', Decimal('6.9971'), 686, Decimal('48')), ('it', Decimal('44.5652'), 92, Decimal('41')), ('pt', Decimal('37.3626'), 182, Decimal('68'))]
AirFrance
[('ar', Decimal('60.0000'), 5, Decimal('3')), ('de', Decimal('31.2500'), 16, Decimal('5')), ('en', Decimal('38.5212'), 5545, Decimal('2136')), ('fr', Decimal('16.4514'), 4103, Decimal('675')), ('it', Decimal('29.7297'), 74, Decimal('22')), ('pt', Decimal('9.1667'), 240, Decimal('22'))]
British_Airways
[('ar', Decimal('9.2308'), 65, Decimal('6')), ('de', Decimal('9.0909'), 44, Decimal('4')), ('en', Decimal('45.9819'), 83372, Decimal('38336')), ('fr', Decimal('11.4754'), 61, Decimal('7')), ('hi', Decimal('0.0000'), 2, Decimal('0')), ('it', Decimal('6.0606'), 33, Decimal('2')), ('pt', Decimal('13.7931'), 29, Decimal('4'))]
AmericanAir
[('ar', Decimal('0.0000'), 43, Decimal('0')), ('de', Deci

In [11]:
# create the dataframe to store the language counts
df = pd.DataFrame(columns=['language', 'sentiment'])

# get all the sentiment scores per language
for language in jh.languages_list:
    query = """
    SELECT sentiment
    FROM tweets
    WHERE language = %s
    """
    cursor.execute(query, (language,))
    sentiments = cursor.fetchall()
    sentiment_list = [sentiment[0] for sentiment in sentiments]

    # add the language and the sentiment to the dataframe
    df.loc[len(df.index)] = [language, sentiment_list]

df.head()

,language,sentiment
0,ar,"[-0.0234151, -0.0694676, -0.00520112, -0.35635..."
1,en,"[-0.308782, 0.549322, 0.864803, -0.792114, 0.2..."
2,fr,"[-0.873595, -0.873595, -0.873595, -0.873595, -..."
3,pt,"[0.554107, 0.127242, 0.0215067, -0.3548, 0.032..."
4,sp,[]


In [12]:
# Close the cursor and connection
cursor.close()
connection.close()
